In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier # 분류 알고리즘 (비지도/지도학습 중 지도학습 안에 분류/회귀)
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()

# 데이터셋을 분할하지 않고 예측하는 경우(전체 데이터를 학습, test_size 0%)
train_data = iris.data
train_label = iris.target

dt_clf.fit(train_data, train_label)
pred = dt_clf.predict(train_data) # 학습 데이터로 예측 진행

print('predictioin accuracy:', accuracy_score(train_label, pred))

# 데이터를 분할하지 않으면 학습/예측의 의미가 없음(정확도 1.0; 100%)
# 테스트 데이터로 예측을 해주어야 함

predictioin accuracy: 1.0


# cross_val_score

- 교차검증을 쉽게 하도록 도와주는 API
- 폴드 세트 추출, 학습/예측, 평가를 한 번에 수행 가능

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris

iris = load_iris()

# feature 데이터와 정답 데이터를 변수에 저장
data = iris.data # feature
label = iris.target # target

dt_clf = DecisionTreeClassifier(random_state=156) # 알고리즘 객체 생성

## cross_val_score 이용한 교차검증

In [3]:
# cross_val_score import
from sklearn.model_selection import cross_val_score
import numpy as np

In [4]:
# cross_val_score의 파라미터

# estimator: 구현하고자 하는 모델/어떤 알고리즘을 사용할 것인지(분류, 회귀)
# X(대문자): 데이터 세트
# y(소문자): label 데이터 세트
# scoring: 검증 지표(성능 평가 지표)
# cv: 교차 검증의 fold 숫자(몇 번의 교차 검증을 진행할 것인지)

scores = cross_val_score(estimator=dt_clf,
                        X=data, y=label,
                        cv=3, scoring='accuracy')

# 'Fold val accuracy:' 교차검증한 결과값을 출력
print('Fold val accuracy:', scores)

# 'Avg val accuracy:' 교차검증한 결과값의 평균을 출력 -> 최종 accuracy 값
print('Avg val accuracy:', np.round(np.mean(scores), 3))

# 교차검증을 한다고 해서 꼭 정확도가 더 높게 나오는 것은 아님
# 머신러닝의 목표는 테스트 데이터에서도 좋은 성능을 유지하는 모델을 만드는 것
# 따라서 이런저런 시도를 많이 해볼 필요가 있음

Fold val accuracy: [0.98 0.94 0.98]
Avg val accuracy: 0.967


## GridSearchCV
- 알고리즘에 파라미터를 삽입하여 파라미터에 맞게 여러 번 교차검증을 진행

In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()

# 데이터셋을 x_train, x_test, y_train, y_test로 분리, test_size: 20%
x_train, x_test, y_train, y_test = train_test_split(iris.data,
                                                   iris.target,
                                                   test_size=0.2,
                                                   random_state=11)

In [6]:
# x_train, y_train의 크기 출력
x_train.shape, y_train.shape

((120, 4), (120,))

In [7]:
from sklearn.tree import DecisionTreeClassifier # 알고리즘 선택
from sklearn.model_selection import GridSearchCV # 모든 검증 함수는 model_selection 안에 존재

dtree = DecisionTreeClassifier() # 알고리즘 객체 생성
parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2, 3]} # dictionary 형태


# GridSearchCV의 파라미터

# estimator: 구현하고자 하는 모델
# param_grid: 모델의 튜닝에 사용될 파라미터 정보(dictionary 형태)
# scoring: 검증 지표(성능 평가 지표)
# cv: 교차 검증의 fold 숫자
# refit: 최적의 파라미터로 모델 재학습 여부(default: True)

# GridSearchCV 교차검증
grid_dtree = GridSearchCV(dtree, # 첫 번째 인자 default: estimator
                         param_grid=parameters,
                         cv=3, refit=True)

# scoring defalut for classification -> sklearn.metrics.accuracy_score
# scoring default for regression -> sklearn.metrics.r2_score


# 모델 학습
# return 값: grid_dtree.cv_results_(결과값이 저장되는 변수 cv_results_)
grid_dtree.fit(x_train, y_train)

# 교차 검증 없이 학습을 진행한 경우:
# dtree 객체 선언 -> dtree.fit(x_train, y_train)

# GridSearchCV 교차 검증 진행한 경우:
# dtree 객체 선언 -> grid_dtree = GridSearchCV(dtree, ...) -> grid_dtree.fit(x_train, y_train)
# 기존에 선언된 객체에 교차 검증 함수를 한 번 더 씌워서 진행

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

In [8]:
import pandas as pd

# grid_dtree.cv_results_ 값을 scores_df(데이터프레임)에 저장
scores_df = pd.DataFrame(grid_dtree.cv_results_)

# scores_df의 확인하고 싶은 column만 추출
scores_df[['params', 'mean_test_score', 'rank_test_score',
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

# max_depth 3, min_samples_split 2 or 3일 때 best_score

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.675000,5,0.675,0.675,0.675
1,"{'max_depth': 1, 'min_samples_split': 3}",0.675000,5,0.675,0.675,0.675
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.000,0.950
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.000,0.950
4,"{'max_depth': 3, 'min_samples_split': 2}",0.966667,1,0.925,1.000,0.975
5,"{'max_depth': 3, 'min_samples_split': 3}",0.966667,1,0.925,1.000,0.975


In [9]:
# 최적의 파라미터, 최고의 정확도 확인
print('Optimal parameter:', grid_dtree.best_params_) # best_params_: 최적의 파라미터 저장
print(f'Max accuracy: {grid_dtree.best_score_:.4f}') # best_score_: 최고 스코어 저장

Optimal parameter: {'max_depth': 3, 'min_samples_split': 2}
Max accuracy: 0.9667


In [10]:
from sklearn.metrics import accuracy_score # 성능측정과 관련된 함수는 대부분 metrics 안에 존재

# 테스트 데이터로 예측 수행
pred = grid_dtree.predict(x_test)

# 정확도 측정값을 accur에 저장
accur = accuracy_score(y_test, pred) # test의 정답 데이터와 예측 결과를 비교
print(f'Test dataset accuracy: {accur:.4f}')

# 학습 데이터 Max accuracy 0.9667, 테스트 데이터 accuracy 0.9333
# 테스트 데이터도 비슷한 성능 반환 -> 교차 검증이 잘 이루어짐

# 데이터가 너무 적거나 과적합의 가능성이 있는 경우,
# 즉 학습 데이터의 accuracy는 높으나 테스트 데이터의 accuracy가 현저히 낮은 경우,
# 교차 검증을 통해 최적의 파라미터를 찾아보는 것이 좋은 방법

Test dataset accuracy: 0.9333
